# 🐳 Docker für Data Science – Deep Dive & Production Ready
 
> 🚀 **Motivation für Entwickler:**
> 
> Du kennst "It works on my machine"? Docker beendet das.
> Wir bauen hier nicht nur irgendeinen Container, sondern eine **Production-Ready Environment** mit Security-Best-Practices, Caching und Orchestrierung.
 
**Ziel:** Du verstehst Linux-Namespaces, Layer-Caching und wie du eine Data-Science-App sicher deployest.

---

## 🤔 Was ist Docker wirklich?

Docker ist mehr als nur eine "leichte VM". Es ist **Prozess-Isolation** auf Kernel-Ebene.

### 🧠 Deep Dive: Die Technik dahinter
- **Namespaces:** Isolieren was der Prozess *sieht* (PIDs, Netzwerk, Mounts). Dein Container denkt, er ist allein auf der Welt.
- **Cgroups (Control Groups):** Limitieren was der Prozess *nutzen darf* (CPU, RAM). Verhindert, dass ein Container den Server lahmlegt.
- **UnionFS (OverlayFS):** Ein Schicht-basiertes Dateisystem. Änderungen werden als neue Schicht oben drauf gelegt (Copy-on-Write). Das macht Images so effizient.

## 🏗️ Docker Grundbegriffe

### 📦 Container
- Eine "Box" mit allem, was deine App braucht
- Läuft isoliert vom Rest des Systems
- Kann gestartet, gestoppt und gelöscht werden

### 🗂️ Image (Abbild)
- Die "Vorlage" für einen Container
- Wie ein Rezept oder eine Bauanleitung
- Unveränderlich und wiederverwendbar

### 📝 Dockerfile
- Eine Textdatei mit Anweisungen
- Beschreibt, wie ein Image gebaut wird
- Wie eine Schritt-für-Schritt Anleitung

### 🎼 Docker Compose
- Orchestriert mehrere Container
- Wie ein Dirigent für ein Orchester
- Startet alle Services mit einem Befehl

## 🎯 Warum Docker für Data Science?

### ✅ Vorteile

1. **Reproduzierbarkeit** 🔄
   - Deine Analyse läuft überall gleich
   - Keine "Bei mir funktioniert es"-Probleme

2. **Einfache Installation** ⚡
   - Ein Befehl startet alles
   - Keine komplizierte Setup-Prozedur

3. **Isolation** 🏠
   - Verschiedene Projekte stören sich nicht
   - Keine Versionskonflikte

4. **Teamarbeit** 👥
   - Alle haben die gleiche Umgebung
   - Einfaches Teilen von Projekten

5. **Deployment** 🚀
   - Apps einfach in die Cloud bringen
   - Von lokal zu produktiv ohne Änderungen

## 🛠️ Docker Installation

### Windows & Mac
1. Gehe zu [docker.com](https://www.docker.com/products/docker-desktop)
2. Lade Docker Desktop herunter
3. Installiere und starte es
4. Teste mit: `docker --version`

### Linux (Ubuntu/Debian)
```bash
# Docker installieren
curl -fsSL https://get.docker.com -o get-docker.sh
sudo sh get-docker.sh

# Docker Compose installieren
sudo apt install docker-compose

# Benutzer zur Docker-Gruppe hinzufügen
sudo usermod -aG docker $USER
```

In [1]:
%%writefile Dockerfile
# 🏗️ Production-Ready Dockerfile
# 1. Basis-Image: Slim Variante für minimale Angriffsfläche & Größe
FROM python:3.11-slim

# 2. Best Practices: Bytecode verhindern & Output unbuffered (für Logs)
ENV PYTHONDONTWRITEBYTECODE=1 \
    PYTHONUNBUFFERED=1

# 3. Arbeitsverzeichnis setzen
WORKDIR /app

# 4. System-Dependencies installieren & Cache bereinigen (Layer-Optimierung!)
# Wir installieren curl für Healthchecks
RUN apt-get update && apt-get install -y \
    curl \
    && rm -rf /var/lib/apt/lists/*

# 5. Security: Nicht als Root laufen!
# Wir erstellen einen dedizierten User 'appuser'
RUN useradd -m -u 1000 appuser
USER appuser

# 6. Python-Dependencies installieren
# Erst requirements kopieren (nutzt Docker Layer Caching wenn sich Code ändert aber reqs nicht)
COPY --chown=appuser:appuser requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# 7. Code kopieren
COPY --chown=appuser:appuser . .

# 8. Metadaten & Healthcheck
EXPOSE 8501
HEALTHCHECK CMD curl --fail http://localhost:8501/_stcore/health || exit 1

# 9. Start-Befehl
CMD ["streamlit", "run", "uebungs_app.py", "--server.address", "0.0.0.0", "--server.port", "8501"]

Overwriting Dockerfile


In [2]:
%%writefile docker-compose.yml
version: '3.8'

services:
  # Unsere Data Science App
  app:
    build: .
    container_name: data-science-pro
    restart: unless-stopped  # Auto-Restart bei Crash
    ports:
      - "8501:8501"
    volumes:
      - .:/app              # Hot-Reloading für Entwicklung
    environment:
      - ENV=development
    # Ressourcen-Limits (Cgroups in Action)
    deploy:
      resources:
        limits:
          cpus: '1.0'
          memory: 1G

Overwriting docker-compose.yml


In [3]:
%%writefile requirements.txt
streamlit==1.52.1
pandas==2.2.3
plotly==5.24.1
numpy==1.26.4
scikit-learn==1.6.0
matplotlib==3.10.0
seaborn==0.13.2
mlflow==2.19.0
opentelemetry-api==1.29.0
opentelemetry-sdk==1.29.0

Overwriting requirements.txt


## 🚀 Starten & Testen

Jetzt starten wir unseren optimierten Stack:

```bash
# 1. Image bauen und Container starten
docker-compose up --build -d

# 2. Logs checken (wichtig für Debugging)
docker-compose logs -f

# 3. Status prüfen
docker-compose ps
```

👉 Öffne **http://localhost:8501** im Browser.

### 🛠️ Troubleshooting Profi-Tipps
- **Rebuild erzwingen:** `docker-compose up --build` (wenn du neue Pip-Pakete hast).
- **Alles aufräumen:** `docker system prune` (löscht ungenutzte Images/Container).
- **In den Container gehen:** `docker exec -it data-science-pro bash` (für manuelles Debugging).

## 🌟 Docker Best Practices

### 1. Kleine Images verwenden
- Alpine Linux Images (klein und sicher)
- Nur notwendige Pakete installieren

### 2. Volumes für Daten
- Daten außerhalb des Containers speichern
- Überleben Container-Neustarts

### 3. Environment Variables
- Konfiguration flexibel halten
- Geheimnisse sicher verwalten

### 4. .dockerignore verwenden
- Unnötige Dateien ausschließen
- Build-Zeit reduzieren
- **Security:** Verhindert, dass `.env` Files oder Secrets im Image landen!

### 5. Multi-Stage Builds
- Verschiedene Umgebungen (dev, test, prod)
- Kleinere finale Images

### 6. Security First
- **Niemals als Root:** Nutze `USER appuser`
- **Minimal Base Images:** `python:slim` oder `alpine` (Vorsicht bei C-Libs)

## 🎓 Zusammenfassung

**Was du gelernt hast:**
- ✅ Was Docker ist und warum es nützlich ist
- ✅ Grundlegende Docker-Begriffe
- ✅ Installation und Setup
- ✅ Wichtige Kommandos
- ✅ Troubleshooting-Tipps
- ✅ Best Practices

**Nächste Schritte:**
1. Docker installieren und testen
2. Unser docker-compose Setup ausprobieren
3. Mit Jupyter Lab erste Python-Schritte machen
4. Erste Streamlit-App entwickeln

## 🏆 Pro-Quiz: Docker Internals

### Frage 1: Layer Caching
Warum kopieren wir `requirements.txt` VOR dem restlichen Code ins Dockerfile?
A) Weil es schöner aussieht.
B) Damit Docker den Cache für die Pip-Installation nutzen kann, auch wenn sich der App-Code ändert.

### Frage 2: Volumes vs. Copy
Was ist der Unterschied zwischen `COPY . .` und einem Volume `./:/app`?
A) `COPY` brennt den Code ins Image (für Production), Volumes spiegeln Live-Änderungen (für Dev).
B) Volumes sind langsamer.

### Frage 3: Security
Warum sollte man `USER appuser` nutzen?
A) Damit man Zugriff auf Root-Dateien hat.
B) Damit ein Angreifer bei einem Hack nicht automatisch Root-Rechte auf dem Host-System hat.

---

**💡 Tipp**: Docker scheint am Anfang kompliziert, aber es macht dein Leben als Data Scientist viel einfacher! Dranbleiben lohnt sich! 🚀